<a href="https://colab.research.google.com/github/AyushiKashyapp/indian_railways_network/blob/main/2_Neo4j_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating a Neo4j Knowledge Graph using the extracted triples.
A knowledge graph created using the triples extracted in the previous parts of the project, sources from the wikipedia pages and web pages of the major stakeholders and committess members of the Food Wise and Food Vision projects.

Installing required libraries

xlrd : To import .xlsx file, which stores the extracted triples.
neo4j-driver : To establish connection with Neo4j Sandbox instance.

In [1]:
!pip install xlrd
!pip install neo4j-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.22.0-py3-none-any.whl size=293755 sha256=23d9bf047ce4735dc273c48ebc16619f60e5cdf9fbfa96c69f4f63a6f5ae99cf
  Stored in directory: /root/.cache/pip/wheels/aa/00/c4/5fab596e1d9061c4858e7e70682836fc03e19306ecdcb4a16a
Successfully built neo4j-driver


In [3]:
import pandas as pd
import re

**Reading the excel file that contains the triples, and stroing them in a dataframe 'triples'.**

In [4]:
triples = pd.read_excel('/content/train_station_triples.xlsx')
triples.head()

,Subject,Predicate,Object
0,Badhal,rdf:type,station
1,Badhal,state,Rajasthan
2,Badhal,code,BDHL
3,Badhal,zone,NWR
4,Badhal,address,"Kishangarh Renwal, Rajasthan"


**Triple relations cleaning**: Replace any non-alphanumeric characters with underscores.

In [2]:
def sanitize_relation_type(relation):
    return re.sub(r'\W+', '_', relation)

**Filter the data to only include a subset of the nodes and edges.**

Subsetting the data for demonstration since the complete triple set contains 74,000+ triples.

In [12]:
def filter_relations(relations, limit=1000):
    filtered_relations = relations[:limit]
    return filtered_relations

**Creating the knowledge database using Neo4j driver and with the head, cleanned relation text , and tail of the triple.**

In [5]:

from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.235.130.12:7687",
  auth=basic_auth("neo4j", "fittings-existence-billets"))

cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()

<ipython-input-5-8972dd92e129>:14: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


0


In [13]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.235.130.12:7687",
  auth=basic_auth("neo4j", "fittings-existence-billets"))

def create_knowledge_graph(triples, limit):

    triple_subset = filter_relations(triples, limit=limit)
    triple_subset.dropna(subset=['Subject', 'Object'], inplace=True)

    with driver.session() as session:
        for _, triple in triple_subset.iterrows():
            Subject = triple['Subject']
            Predicate = sanitize_relation_type(triple['Predicate']) #Applying text cleaning function on the relations in triples.
            Object = triple['Object']

            cypher_query = f"""
            MERGE (h:Entity {{name: $Subject}})
            MERGE (t:Entity {{name: $Object}})
            MERGE (h)-[:{Predicate.upper()}]->(t)
            """

            session.run(cypher_query, Subject=Subject, Object=Object)


In [15]:
create_knowledge_graph(triples, 1000)

<ipython-input-13-60dc75867112>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  triple_subset.dropna(subset=['Subject', 'Object'], inplace=True)


In [16]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.235.130.12:7687",
  auth=basic_auth("neo4j", "fittings-existence-billets"))

cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()

14194


<ipython-input-16-8972dd92e129>:14: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(
